In [4]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from numpy import dot
from numpy.linalg import norm
import multiprocessing as mp

OUTPUT = "/home/twalton_umass_edu/Political Polarization Project/tmls/word_embeddings/clustering/"
INPUT = '/home/twalton_umass_edu/Political Polarization Project/tmls/word_embeddings/sent_embed/'
NUM_CLUSTERS = 40

#create list of numbers to represent months
months = range(8)
#load model
model = Word2Vec.load('/home/twalton_umass_edu/Political Polarization Project/tmls/word_embeddings/dem_word2vec_100.model')
#load means
means = np.load('/home/twalton_umass_edu/Political Polarization Project/tmls/word_embeddings/clustering/dem_cluster_' + str(NUM_CLUSTERS) + '_means.npy')

#cosine similarity function
def cosine_sim(a, b):
    return dot(a, b) / (norm(a)*norm(b))

In [5]:
#function that gets the scores/saves for each month
def get_scores(m):
    
    #load cleaned text dataframe
    text = pd.read_csv(INPUT + 'cleaned/dem_clean_' + str(m) + '.csv')
    #load sentence embeddings
    embedding = pd.read_csv(INPUT + 'trained/dem_trained_' + str(m) + '.csv')
    
    print('length of text: ' + str(len(text)))
    print('length of embedding: ' + str(len(embedding)))
    
    #remove unnamed axxis from embedding
    embedding = embedding.drop(['Unnamed: 0'], axis = 1)
    
    dicts = []
    
    for embed in range(len(embedding)):
        d = {}
        distances = np.array([cosine_sim(embedding.loc[embed], me) for me in means])
        sorted_dists = distances.argsort()
        for i, topic in enumerate(sorted_dists):
            d['topic_' + str(i)] = topic  # ith closest topic
            d['cosine_' + str(i)] = distances[topic]  # cosine distance of ith closest topic
        dicts.append(d)
    df = pd.DataFrame(dicts)
    df.to_csv(OUTPUT + 'distances/dem_kmeans_topics_' + str(NUM_CLUSTERS) + '_' + str(m) + '.csv', index=False)

In [ ]:
pool = mp.Pool(mp.cpu_count())

results = pool.map(get_scores, [m for m in months])

pool.close()

length of text: 401951
length of embedding: 401951
length of text: 608384
length of embedding: 608384
length of text: 596779
length of embedding: 596779
length of text: 517796
length of embedding: 517796
length of text: 561750
length of embedding: 561750
length of text: 582628
length of embedding: 582628
length of text: 580913
length of embedding: 580913
length of text: 654338
length of embedding: 654338
